In [90]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import math
import numpy as np
import tensorflow as tf
from PIL import Image
from matplotlib import pyplot as plt

# local python package
sys.path.append(os.path.join(os.getcwd(), '..'))

from voc_loader import VOC_loader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def plot_result(img, gt_seg, pred_seg, palette):
    plt.imshow(img.astype(np.uint8))
    plt.imshow(get_seg.astype(np.uint8))
    
    np.place(pred_seg, pred_seg==21, 255)
    tmp = Image.fromarray(pred_seg, 'P')
    tmp.putpalette(palette)
    plt.imshow(tmp)
    plt.show()

## Download VOC 2012 dataset and create loader

In [83]:
if not os.path.exists('VOCdevkit'):
    os.system('wget http://cvlab.postech.ac.kr/~jonghwan/VOC2012.tar')
    os.system('tar xvf VOC2012.tar')
    os.system('rm VOC2012.tar')

In [91]:
loader_params = {
    'num_classes': 22,
    'image_size': 448,
    'split_root': 'VOCdevkit/VOC2012/ImageSets/Segmentation',
    'image_root': 'VOCdevkit/VOC2012/JPEGImages',
    'segmap_root': 'VOCdevkit/VOC2012/SegmentationClass',
}
loader = VOC_loader(loader_params)

In [92]:
batch = loader.get_batch(3, 'train')